<a href="https://colab.research.google.com/github/hospino/Cursos-Platzi/blob/main/TrainingLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# !pip install torch accelerate transformers peft trl bitsandbytes datasets
# Asegúrate de que el runtime de Colab está configurado a T4 GPU.
# Importaciones necesarias
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [5]:

# 1. Cargar el dataset (ejemplo: IMDB)
dataset = load_dataset("imdb")

# 2. Cargar el tokenizer del LLM base (¡MODELO ACTUALIZADO!)
model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Ajuste común para Mistral

# 3. Función de tokenización
def tokenize_function(examples):
    # Trunca el texto para que no sea demasiado largo para la GPU
    return tokenizer(examples["text"], truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 4. Preparar datos para entrenamiento
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(50))

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:

# 1. Cargar el modelo base para Clasificación de Secuencias
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=2, # Número de clases (positivo/negativo)
    torch_dtype=torch.bfloat16 # Usar bfloat16 para optimizar la memoria en la T4 GPU
)
# Indicar al modelo que use el token de padding que definimos
model.config.pad_token_id = tokenizer.pad_token_id

# 2. Definir la configuración LoRA (PEFT)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"], # Módulos de atención en Mistral
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
)

# 3. Aplicar LoRA al modelo
model = get_peft_model(model, lora_config)

# Opcional: Mostrar cuántos parámetros entrenaremos
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 3,416,064 || all params: 7,114,084,352 || trainable%: 0.0480


In [7]:

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,               # Número de veces que el modelo verá el dataset
    per_device_train_batch_size=4,    # Tamaño del lote para la GPU
    per_device_eval_batch_size=4,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",            # Evaluar después de cada época
    save_strategy="epoch",
    load_best_model_at_end=True,      # Guardar el mejor modelo según la métrica
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    # No es necesario calcular métricas complejas para esta demostración inicial
)

/tmp/ipython-input-4063718027.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

# 1. Ejecutar el Fine-Tuning
trainer.train()

# 2. Prueba del modelo
text = "Este es el peor guion que he visto, una completa pérdida de tiempo."

inputs = tokenizer(text, return_tensors="pt").to("cuda")

with torch.no_grad():
    logits = model(**inputs).logits

# Obtener la clase predicha (0 o 1)
predicted_class_id = logits.argmax().item()
prediction_label = "Positivo" if predicted_class_id == 1 else "Negativo"

print(f"\nTexto: {text}")
print(f"Predicción del modelo Fine-Tuned: {prediction_label}")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: